# Using Tools and Structured Outputs with Gemini

This notebook explores two powerful features for building capable AI agents with Large Language Models (LLMs): **Tools (Function Calling)** and **Structured Outputs**. We will use the `google-genai` library to interact with Google's Gemini models.

**Learning Objectives:**

1.  **Understand and implement tool use (function calling)** to allow an LLM to interact with external systems.
2.  **Enforce structured data formats (JSON)** from an LLM for reliable data extraction.
3.  **Leverage Pydantic models** to define and manage complex data structures for both function arguments and structured outputs, improving code robustness and clarity.

## 1. Setup

First, let's install the necessary Python libraries.

!pip install -q google-generativeai pydantic python-dotenv

### Configure Gemini API Key

To use the Gemini API, you need an API key. 

1.  Get your key from [Google AI Studio](https://aistudio.google.com/app/apikey).
2.  Create a file named `.env` in the root of this project.
3.  Add the following line to the `.env` file, replacing `your_api_key_here` with your actual key:
    ```
    GEMINI_API_KEY="your_api_key_here"
    ```
The code below will load this key from the `.env` file.

In [76]:
import os

from pathlib import Path
from dotenv import load_dotenv


REPOSITORY_ROOT_DIR = Path().absolute().parent.parent
DOTENV_FILE_PATH = REPOSITORY_ROOT_DIR / ".env"
print(f"Trying to load environment variables from `{DOTENV_FILE_PATH}`")

if not DOTENV_FILE_PATH.exists():
    raise FileNotFoundError(f"Environment file `{DOTENV_FILE_PATH}` not found.")

load_dotenv(dotenv_path=DOTENV_FILE_PATH)

assert "GOOGLE_API_KEY" in os.environ, "`GOOGLE_API_KEY` is not set"

print("Environment variables loaded successfully.")

Trying to load environment variables from `/Users/pauliusztin/Documents/01_projects/TAI/course-ai-agents/.env`
Environment variables loaded successfully.


In [109]:
import json

from google import genai
from pydantic import BaseModel, Field
from google.genai import types

### Initialize the Gemini Client

In [77]:
client = genai.Client()

### Define Constants

We will use the `gemini-2.5-flash` model, which is fast, cost-effective, and supports advanced features like tool use.

In [78]:
MODEL_NAME = "gemini-2.5-flash"

## 2. Implementing structured outputs from scratch using JSON

Sometimes, you don't need the LLM to take an action, but you need its output in a specific, machine-readable format. Forcing the output to be JSON is a common way to achieve this.

We can instruct the model to do this by:
1.  **Prompting**: Clearly describe the desired JSON structure in the prompt.
2.  **Configuration**: Setting `response_mime_type` to `"application/json"` in the generation configuration, which forces the model's output to be a valid JSON object.

### Example: Extracting Metadata from a Document

Let's imagine we have a markdown document and we want to extract key information like a summary, tags, and keywords into a clean JSON object.

In [102]:
document = """
# Article: The Rise of AI Agents

This article discusses the recent advancements in AI, focusing on autonomous agents. 
to perform complex, multi-step tasks. Key topics include the ReAct framework, 
We explore how Large Language Models (LLMs) are moving beyond simple text generation 
the importance of tool use, and the challenges of long-term planning. The future 
of software development may be significantly impacted by these new AI paradigms.
"""

prompt = f"""
Please analyze the following document and extract metadata from it. 
The output must be a single, valid JSON object with the following structure:
{{ "summary": "A concise summary of the article.", "tags": ["list", "of", "relevant", "tags"], "keywords": ["list", "of", "key", "concepts"] }}

Document:
--- 
{document}
--- 
"""

# Configure the model to output JSON
config = types.GenerateContentConfig(response_mime_type="application/json")

response = client.models.generate_content(
    model="gemini-2.0-flash", contents=prompt, config=config
)

print("--- Raw LLM Output ---")
print(response.text)

# You can now reliably parse the JSON string
metadata_obj = json.loads(response.text)

print("\n--- Parsed JSON Object ---")
print(metadata_obj)

--- Raw LLM Output ---
{
  "summary": "The article explores recent advancements in AI agents, focusing on their ability to perform complex tasks. It highlights the ReAct framework, the importance of tool use, and the challenges of long-term planning in the context of Large Language Models.",
  "tags": ["AI Agents", "Autonomous Agents", "Large Language Models", "LLMs", "ReAct Framework", "Tool Use", "Long-Term Planning"],
  "keywords": ["AI", "agents", "LLMs", "ReAct", "planning", "autonomous", "software development"]
}

--- Parsed JSON Object ---
{'summary': 'The article explores recent advancements in AI agents, focusing on their ability to perform complex tasks. It highlights the ReAct framework, the importance of tool use, and the challenges of long-term planning in the context of Large Language Models.', 'tags': ['AI Agents', 'Autonomous Agents', 'Large Language Models', 'LLMs', 'ReAct Framework', 'Tool Use', 'Long-Term Planning'], 'keywords': ['AI', 'agents', 'LLMs', 'ReAct', 'pla

## 3. Implementing structured outputs from scratch using Pydantic

While prompting for JSON is effective, it can be fragile. A more robust and modern approach is to use **Pydantic**. Pydantic allows you to define data structures as Python classes. This gives you:

- **A single source of truth**: The Pydantic model defines the structure.
- **Automatic schema generation**: You can easily generate a JSON Schema from the model.
- **Data validation**: You can validate the LLM's output against the model to ensure it conforms to the expected structure and types.

Let's recreate the previous example using Pydantic.

In [104]:
class DocumentMetadata(BaseModel):
    """A class to hold structured metadata for a document."""

    summary: str = Field(description="A concise, 1-2 sentence summary of the document.")
    tags: List[str] = Field(
        description="A list of 3-5 high-level tags relevant to the document."
    )
    keywords: List[str] = Field(
        description="A list of specific keywords or concepts mentioned."
    )

### Injecting Pydantic Schema into the Prompt

We can generate a JSON Schema from our Pydantic model and inject it directly into the prompt. This is a more formal way of telling the LLM what structure to follow.

In [ ]:
schema = DocumentMetadata.model_json_schema()

prompt = f"""
Please analyze the following document and extract metadata from it. 
The output must be a single, valid JSON object that conforms to the following JSON Schema:
```json
{json.dumps(schema, indent=2)}
```

Document:
--- 
{document}
--- 
"""

config = types.GenerateContentConfig(response_mime_type="application/json")
response = client.models.generate_content(
    model=MODEL_NAME, contents=prompt, config=config
)

print("--- Raw LLM Output ---")
print(response.text)

# Now, we can validate the output with Pydantic
try:
    document_metadata = DocumentMetadata.model_validate_json(response.text)
    print("\n--- Pydantic Validated Object ---")
    print(document_metadata)
    print("\nValidation successful!")
except Exception as e:
    print(f"\nValidation failed: {e}")

--- Raw LLM Output ---
{
  "summary": "This article discusses recent advancements in AI, focusing on autonomous agents capable of performing complex, multi-step tasks. It explores how Large Language Models (LLMs) are evolving beyond simple text generation through frameworks like ReAct, emphasizing tool use and addressing challenges in long-term planning, potentially impacting future software development.",
  "tags": [
    "AI",
    "Autonomous Agents",
    "Large Language Models",
    "Software Development",
    "Advanced AI"
  ],
  "keywords": [
    "AI Agents",
    "ReAct framework",
    "Large Language Models",
    "LLMs",
    "Tool use",
    "Long-term planning",
    "Multi-step tasks"
  ]
}

--- Pydantic Validated Object ---
summary='This article discusses recent advancements in AI, focusing on autonomous agents capable of performing complex, multi-step tasks. It explores how Large Language Models (LLMs) are evolving beyond simple text generation through frameworks like ReAct, emp

## 4. Implementing structured ouputs using Gemini and Pydantic

In [ ]:
config = types.GenerateContentConfig(
    response_mime_type="application/json", response_schema=DocumentMetadata
)

prompt = f"""
Please analyze the following document and extract its metadata.

Document:
--- 
{document}
--- 
"""

response = client.models.generate_content(
    model=MODEL_NAME, contents=prompt, config=config
)
response.parsed

DocumentMetadata(summary='This article explores the advancements in AI, specifically autonomous agents performing complex tasks. It highlights the ReAct framework, LLMs, tool use, and challenges in long-term planning, suggesting a significant impact on software development.', tags=['AI Agents', 'Large Language Models', 'Autonomous Systems', 'Software Development'], keywords=['AI', 'autonomous agents', 'ReAct framework', 'LLMs', 'tool use', 'long-term planning'])

## 5. Implementing tool calls from scratch

LLMs are trained on text and can't perform actions in the real world on their own. **Tools** (or **Function Calling**) are the mechanism we use to bridge this gap. We provide the LLM with a list of available tools, and it can decide which one to use and with what arguments to fulfill a user's request.

The process is a loop:
1.  **You**: Send the LLM a prompt and a list of available tools.
2.  **LLM**: Responds with a `function_call` request, specifying the tool and arguments.
3.  **You**: Execute the requested function in your code.
4.  **You**: Send the function's output back to the LLM.
5.  **LLM**: Uses the tool's output to generate a final, user-facing response.

### Define Mock Tools

Let's create two simple, mocked functions. One simulates searching Google Drive, and the other simulates sending a Discord message. The function docstrings are crucial, as the LLM uses them to understand what each tool does.

In [79]:
def search_google_drive(query: str) -> str:
    """
    Searches for a file on Google Drive and returns its content or a summary.

    Args:
        query (str): The search query to find the file, e.g., 'Q3 earnings report'.

    Returns:
        str: A JSON string representing the search results, including file names and summaries.
    """

    # In a real scenario, this would interact with the Google Drive API.
    # Here, we mock the response for demonstration.
    if "q3 earnings report" in query.lower():
        return json.dumps(
            {
                "files": [
                    {
                        "name": "Q3_Earnings_Report_2024.pdf",
                        "id": "file12345",
                        "summary": "The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, beating expectations.",
                    }
                ]
            }
        )
    else:
        return json.dumps({"files": []})


def send_discord_message(channel_id: str, message: str) -> str:
    """
    Sends a message to a specific Discord channel.

    Args:
        channel_id (str): The ID of the channel to send the message to, e.g., '#finance'.
        message (str): The content of the message to send.

    Returns:
        str: A JSON string confirming the action, e.g., '{"status": "success"}'.
    """

    # Mocking a successful API call
    return json.dumps(
        {
            "status": "success",
            "channel": channel_id,
            "message_preview": f"{message[:50]}...",
        }
    )


def summarize_financial_report(text: str) -> str:
    """
    Summarizes a financial report.

    Args:
        text (str): The text to summarize.

    Returns:
        str: The summary of the text.
    """

    return "The Q3 earnings report shows an increase in company metrics."

In [80]:
search_google_drive_declaration = {
    "name": "search_google_drive",
    "description": "Searches for a file on Google Drive and returns its content or a summary.",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "The search query to find the file, e.g., 'Q3 earnings report'.",
            }
        },
        "required": ["query"],
    },
}

send_discord_message_declaration = {
    "name": "send_discord_message",
    "description": "Sends a message to a specific Discord channel.",
    "parameters": {
        "type": "object",
        "properties": {
            "channel_id": {
                "type": "string",
                "description": "The ID of the channel to send the message to, e.g., '#finance'.",
            },
            "message": {
                "type": "string",
                "description": "The content of the message to send.",
            },
        },
        "required": ["channel_id", "message"],
    },
}

summarize_financial_report_declaration = {
    "name": "summarize_financial_report",
    "description": "Summarizes a financial report.",
    "parameters": {
        "type": "object",
        "properties": {
            "text": {
                "type": "string",
                "description": "The text to summarize.",
            },
        },
        "required": ["text"],
    },
}

TOOLS = {
    "search_google_drive": {
        "handler": search_google_drive,
        "declaration": search_google_drive_declaration,
    },
    "send_discord_message": {
        "handler": send_discord_message,
        "declaration": send_discord_message_declaration,
    },
    "summarize_financial_report": {
        "handler": summarize_financial_report,
        "declaration": summarize_financial_report_declaration,
    },
}
TOOLS_BY_NAME = {tool_name: tool["handler"] for tool_name, tool in TOOLS.items()}
TOOLS_SCHEMA = [tool["declaration"] for tool in TOOLS.values()]

In [81]:
TOOLS_BY_NAME

{'search_google_drive': <function __main__.search_google_drive(query: str) -> str>,
 'send_discord_message': <function __main__.send_discord_message(channel_id: str, message: str) -> str>,
 'summarize_financial_report': <function __main__.summarize_financial_report(text: str) -> str>}

In [82]:
TOOLS_SCHEMA

[{'name': 'search_google_drive',
  'description': 'Searches for a file on Google Drive and returns its content or a summary.',
  'parameters': {'type': 'object',
   'properties': {'query': {'type': 'string',
     'description': "The search query to find the file, e.g., 'Q3 earnings report'."}},
   'required': ['query']}},
 {'name': 'send_discord_message',
  'description': 'Sends a message to a specific Discord channel.',
  'parameters': {'type': 'object',
   'properties': {'channel_id': {'type': 'string',
     'description': "The ID of the channel to send the message to, e.g., '#finance'."},
    'message': {'type': 'string',
     'description': 'The content of the message to send.'}},
   'required': ['channel_id', 'message']}},
 {'name': 'summarize_financial_report',
  'description': 'Summarizes a financial report.',
  'parameters': {'type': 'object',
   'properties': {'text': {'type': 'string',
     'description': 'The text to summarize.'}},
   'required': ['text']}}]

In [83]:
TOOL_CALLING_SYSTEM_PROMPT = """
You are a helpful AI assistant with access to tools that enable you to take actions and retrieve information to better assist users.

## Tool Usage Guidelines

**When to use tools:**
- When you need information that is not in your training data
- When you need to perform actions in external systems  
- When you need real-time, dynamic, or user-specific data
- When computational operations are required

**Tool selection:**
- Choose the most appropriate tool based on the user's specific request
- If multiple tools could work, select the one that most directly addresses the need
- Consider the order of operations for multi-step tasks

**Parameter requirements:**
- Provide all required parameters with accurate values
- Use the parameter descriptions to understand expected formats and constraints
- Ensure data types match the tool's requirements (strings, numbers, booleans, arrays)

## Tool Call Format

When you need to use a tool, output ONLY the tool call in this exact format:

```tool_call
{{"name": "tool_name", "args": {{"param1": "value1", "param2": "value2"}}}}
```

**Critical formatting rules:**
- Use double quotes for all JSON strings
- Ensure the JSON is valid and properly escaped
- Include ALL required parameters
- Use correct data types as specified in the tool definition
- Do not include any additional text or explanation in the tool call

## Response Behavior

- If no tools are needed, respond directly to the user with helpful information
- If tools are needed, make the tool call first, then provide context about what you're doing
- After receiving tool results, provide a clear, user-friendly explanation of the outcome
- If a tool call fails, explain the issue and suggest alternatives when possible

## Available Tools

<tool_definitions>
{tools}
</tool_definitions>

Remember: Your goal is to be maximally helpful to the user. Use tools when they add value, but don't use them unnecessarily. Always prioritize accuracy and user experience.
"""


Let's try the prompt with a few examples.

In [84]:
USER_PROMPT = """
Can you help me find the latest quarterly report and share key insights with the team?
"""

messages = [TOOL_CALLING_SYSTEM_PROMPT.format(tools=str(TOOLS_SCHEMA)), USER_PROMPT]

response = client.models.generate_content(
    model=MODEL_NAME,
    contents=messages,
)

print(response.text)

```tool_call
{"name": "search_google_drive", "args": {"query": "latest quarterly report"}}
```


In [85]:
USER_PROMPT = """
Please find the Q3 earnings report on Google Drive and send a summary of it to 
the #finance channel on Discord.
"""

messages = [TOOL_CALLING_SYSTEM_PROMPT.format(tools=str(TOOLS_SCHEMA)), USER_PROMPT]

response = client.models.generate_content(
    model=MODEL_NAME,
    contents=messages,
)
response.text

'```tool_call\n{"name": "search_google_drive", "args": {"query": "Q3 earnings report"}}\n```'

The next step, is to parse the LLM response and call the tool using Python.

First, we parse the LLM output to extract the JSON out of the response:

In [86]:
def extract_tool_call(response_text: str) -> str:
    """
    Extracts the tool call from the response text.
    """
    return response_text.split("```tool_call")[1].split("```")[0].strip()


tool_call_str = extract_tool_call(response.text)
tool_call_str

'{"name": "search_google_drive", "args": {"query": "Q3 earnings report"}}'

Next, we parse the stringified JSON to a python dict:

In [87]:
tool_call = json.loads(tool_call_str)
tool_call

{'name': 'search_google_drive', 'args': {'query': 'Q3 earnings report'}}

Now, we retrieve the tool handler, which is a Python function:

In [88]:
tool_handler = TOOLS_BY_NAME[tool_call["name"]]
tool_handler

<function __main__.search_google_drive(query: str) -> str>

Ultimately, we call the Python function using the arguments generated by the LLM:

In [89]:
tool_handler(**tool_call["args"])

'{"files": [{"name": "Q3_Earnings_Report_2024.pdf", "id": "file12345", "summary": "The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, beating expectations."}]}'

That's tool calling in a nutshell!

## 6. Implementing tool calls with Gemini

In production, most of the times, we don't implement tool calling from scratch, but leverage the interface of a specific API such as Gemini or OpenAI. Thus, let's see how we can adapt the example from above to Gemini.

In [99]:
def print_user_prompt(user_prompt) -> None:
    print("=" * 75)
    print(f"User Prompt: {user_prompt}".strip())
    print("=" * 75)


def print_function_call(response) -> None:
    response_message_part = response.candidates[0].content.parts[0]

    if hasattr(response_message_part, "function_call"):
        function_call = response_message_part.function_call
        tool_name = function_call.name
        tool_args = function_call.args

        print("=" * 75)
        print(f"Function Call: {function_call}")
        print(f"Function Name: {tool_name}")
        print(f"Function Arguments: {tool_args}")
        print("=" * 75)
    else:
        print("No function call found in the response.")

In [91]:
tools = [
    types.Tool(
        function_declarations=[
            types.FunctionDeclaration(**search_google_drive_declaration),
            types.FunctionDeclaration(**send_discord_message_declaration),
        ]
    )
]
config = types.GenerateContentConfig(
    tools=tools,
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="ANY")
    ),
)

print_user_prompt(USER_PROMPT)
response = client.models.generate_content(
    model=MODEL_NAME,
    contents=USER_PROMPT,
    config=config,
)
print_function_call(response)

User Prompt: 
Please find the Q3 earnings report on Google Drive and send a summary of it to 
the #finance channel on Discord.
Function Call: id=None args={'query': 'Q3 earnings report'} name='search_google_drive'
Function Name: search_google_drive
Function Arguments: {'query': 'Q3 earnings report'}


In [92]:
response_message_part = response.candidates[0].content.parts[0]
response_message_part.function_call

FunctionCall(id=None, args={'query': 'Q3 earnings report'}, name='search_google_drive')

In [93]:
tool_handler = TOOLS_BY_NAME[response_message_part.function_call.name]
tool_handler

<function __main__.search_google_drive(query: str) -> str>

In [94]:
tool_handler(**response_message_part.function_call.args)

'{"files": [{"name": "Q3_Earnings_Report_2024.pdf", "id": "file12345", "summary": "The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, beating expectations."}]}'

Now let's aggregate everything into a single function:

In [95]:
def call_tool(function_call) -> str:
    tool_name = function_call.name
    tool_args = function_call.args

    tool_handler = TOOLS_BY_NAME[tool_name]

    return tool_handler(**tool_args)


def print_tool_result(tool_result) -> None:
    print("=" * 75)
    print(f"Tool Result: {tool_result}")
    print("=" * 75)

In [96]:
print_tool_result(call_tool(response_message_part.function_call))

Tool Result: {"files": [{"name": "Q3_Earnings_Report_2024.pdf", "id": "file12345", "summary": "The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, beating expectations."}]}


## 7 Implementing tool calls with Gemini: Running tools in a loop

Now, let's see what happens if we put tool calling in a loop. Let's create a scenario where we ask the agent to perform a multi-step task: find a report on Google Drive and then communicate its findings on Discord.

In [101]:
USER_PROMPT = """
Please find the Q3 earnings report on Google Drive and send a summary of it to 
the #finance channel on Discord.
"""

messages = [USER_PROMPT]

print_user_prompt(USER_PROMPT)
response = client.models.generate_content(
    model=MODEL_NAME,
    contents=messages,
    config=config,
)
response_message_part = response.candidates[0].content.parts[0]
print_function_call(response)

messages.append(response.candidates[0].content)

# Loop until a function call is still available or we reach the max number of iterations
max_iterations = 3
while hasattr(response_message_part, "function_call") and max_iterations > 0:
    tool_result = call_tool(response_message_part.function_call)
    print_tool_result(tool_result)

    # Add the tool result to the messages creating the following structure:
    # - user prompt
    # - tool call
    # - tool result
    # - tool call
    # - tool result
    # ...
    function_response_part = types.Part(
        function_response=types.FunctionResponse(
            name=response_message_part.function_call.name,
            response=json.loads(tool_result),
        )
    )
    messages.append(function_response_part)

    response = client.models.generate_content(
        model=MODEL_NAME,
        contents=messages,
        config=config,
    )

    response_message_part = response.candidates[0].content.parts[0]
    messages.append(response.candidates[0].content)

    print_function_call(response)

    max_iterations -= 1

# 4. Print the final, user-facing answer
print("\n==== Final Agent Response ====")
print(response.candidates[0].content)


User Prompt: 
Please find the Q3 earnings report on Google Drive and send a summary of it to 
the #finance channel on Discord.
Function Call: id=None args={'query': 'Q3 earnings report'} name='search_google_drive'
Function Name: search_google_drive
Function Arguments: {'query': 'Q3 earnings report'}
Tool Result: {"files": [{"name": "Q3_Earnings_Report_2024.pdf", "id": "file12345", "summary": "The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, beating expectations."}]}
Function Call: id=None args={'channel_id': '#finance', 'message': 'The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, beating expectations.'} name='send_discord_message'
Function Name: send_discord_message
Function Arguments: {'channel_id': '#finance', 'message': 'The Q3 earnings report shows a 20% increase in revenue and a 15% growth in user engagement, beating expectations.'}
Tool Result: {"status": "success", "channel": "#finance", "messa

As we can see the LLM loop got stuck between retrieving documents from Google Drive and sending them to Discord. To solve this problem, this is where the ReAct algorithm kicks in, which we will discuss in the next lesson.

## 8. Using a Pydantic Model as a Tool for Structured Outputs

A more elegant and powerful pattern is to treat our Pydantic model *as a tool*. We can ask the model to "call" this Pydantic tool, and the arguments it generates will be our structured data.

This combines the power of function calling with the robustness of Pydantic for structured data extraction. It's the recommended approach for complex data extraction tasks.

In [113]:
# The Pydantic class 'DocumentMetadata' is now our 'tool'
extraction_tool = types.Tool(
    function_declarations=[
        types.FunctionDeclaration(
            name="extract_metadata",
            description="Extracts structured metadata from a document.",
            parameters=DocumentMetadata.model_json_schema(),
        )
    ]
)
config = types.GenerateContentConfig(
    tools=[extraction_tool],
    tool_config=types.ToolConfig(
        function_calling_config=types.FunctionCallingConfig(mode="ANY")
    ),
)

prompt = f"""
Please analyze the following document and extract its metadata.

Document:
--- 
{document}
--- 
"""

response = client.models.generate_content(
    model=MODEL_NAME, contents=prompt, config=config
)
response_message_part = response.candidates[0].content.parts[0]

if hasattr(response_message_part, "function_call"):
    function_call = response_message_part.function_call
    print("===== Function Call =====")
    print(function_call)

    try:
        document_metadata = DocumentMetadata(**function_call.args)
        print("\n==== Pydantic Validated Object ====")
        print(document_metadata)
    except Exception as e:
        print(f"\nValidation failed: {e}")
else:
    print("The model did not call the extraction tool.")

===== Function Call =====
id=None args={'keywords': ['AI agents', 'ReAct framework', 'Large Language Models', 'LLMs', 'tool use', 'long-term planning', 'software development'], 'tags': ['Artificial Intelligence', 'Large Language Models', 'Autonomous Agents', 'Software Development', 'Future of Technology'], 'summary': 'This article discusses the recent advancements in AI, focusing on autonomous agents. It explores how Large Language Models (LLMs) are moving beyond simple text generation to perform complex, multi-step tasks.'} name='extract_metadata'

==== Pydantic Validated Object ====
summary='This article discusses the recent advancements in AI, focusing on autonomous agents. It explores how Large Language Models (LLMs) are moving beyond simple text generation to perform complex, multi-step tasks.' tags=['Artificial Intelligence', 'Large Language Models', 'Autonomous Agents', 'Software Development', 'Future of Technology'] keywords=['AI agents', 'ReAct framework', 'Large Language Mode